In [1]:
import numpy as np

In [100]:
class RNN():
    def __init__(self,layers,data,bit_size, sample_size, epoch,alpha):
        self.nr_layers = len(layers)
        self.iLayer_size = layers[0]
        self.oLayer_size = layers[2]
        self.hLayer_size = layers[1]
        self.bit_size = bit_size
        self.sample_size = sample_size
        self.alpha = alpha
        self.epoch = epoch
        
        
        
        self.iSynopse = np.random.randn(self.iLayer_size,self.hLayer_size)
        self.hSynopse = np.random.randn(self.hLayer_size,self.hLayer_size)
        self.oSynopse = np.random.randn(self.hLayer_size,self.oLayer_size)
        
        
    
    def display(self, mode):
        if mode=='i':
            print('iSynopose: {0} '.format(self.iSynopse))
        elif mode=='h':      
            print('hSynopose: {0} '.format(self.hSynopse))
        elif mode=='o':
            print('oSynopose: {0} '.format(self.oSynopse))
        else:    
            print('iLayer_size: {0} hLayer_size {1} oLayer_size {2} '.format(self.iLayer_size,self.hLayer_size, self.oLayer_size) )

            
    def rnn_process(self):
        for epoch in range(self.epoch):
            total_cost = 0.
            for sample in range(len(data)):
                a,b,c =data[sample]
                X = [[a_,b_] for a_,b_ in zip(a,b)]
                Y = c
                iSynopse_update = np.zeros_like(self.iSynopse)
                hSynopse_update = np.zeros_like(self.hSynopse)
                oSynopse_update = np.zeros_like(self.oSynopse)


                hActivations = []
                oActivations = []
                iActivations = []
                oResult = []
                hactivation = np.zeros(self.hLayer_size)
                oactivation = np.zeros(self.oLayer_size)
                cost = np.zeros(self.oLayer_size)
                prev_hactivation = np.atleast_2d(np.zeros(self.hLayer_size))
                temp_cost = 0.
                for i in range(self.bit_size):
                    x = X[-i-1]
                    y = Y[-i-1]
                    hactivation = sigmoid(np.dot(np.atleast_2d(x),self.iSynopse) + np.dot(prev_hactivation, self.hSynopse))
                    #print(prev_hactivation.shape, self.hSynopse.shape, hactivation)
                    hActivations.append(hactivation)
                    oactivation = sigmoid(np.dot(hactivation,self.oSynopse))
                    
                    oActivations.append(oactivation)
                    iActivations.append(x)
                    oResult.append(y)
                    prev_hactivation = hactivation
                    answer = np.heaviside(oactivation, 0.5)
                    cost = np.abs(y-oactivation)
                    #print('y:', y, 'calculation:', answer,'cost', cost)
                    temp_cost += cost
                total_cost+=temp_cost/self.bit_size 
                #print('tot', total_cost)
                o_deltas = []
                h_deltas = []
                future_h_delta = np.atleast_2d(np.zeros(self.hLayer_size))
                
                #print(future_delta.shape)
                for i in range(1,self.bit_size+1):
                    o_delta = (oResult[-i]-oActivations[-i])*sigmoid_der(oActivations[-i])
                    o_deltas.append(o_delta)
                    #print(cost_delta.shape,self.oSynopse.shape)
                    h1_delta = np.dot(o_delta, self.oSynopse.T)*sigmoid_der(hActivations[-i])
                    h2_delta = np.dot(future_h_delta, self.hSynopse.T)*sigmoid_der(hActivations[-i])
                    h_delta = h1_delta + h2_delta
                    h_deltas.append(h_delta)

                    iSynopse_update += np.dot(np.atleast_2d(iActivations[-i]).T, h_delta)
                    hSynopse_update += np.dot(np.atleast_2d(hActivations[-i]).T, future_h_delta)
                    oSynopse_update += np.dot(np.atleast_2d(hActivations[-i]).T, o_delta)

                    future_h_delta = h_delta

                #self.iSynopse +=self.alpha*iSynopse_update/self.bit_size
                #self.hSynopse +=self.alpha*hSynopse_update/self.bit_size
                #self.oSynopse +=self.alpha*oSynopse_update/self.bit_size

                self.iSynopse +=self.alpha*iSynopse_update
                self.hSynopse +=self.alpha*hSynopse_update
                self.oSynopse +=self.alpha*oSynopse_update
                
                
            print('average cost for the epoch {0} is {1}'.format(epoch, total_cost/self.sample_size))    
            if (epoch%100==0):
                print('average cost for the epoch {0} is {1}'.format(epoch, total_cost/self.sample_size))
        
        #print(self.iSynopse.shape, self.iSynopse)
        

            
def sigmoid(z): return (1/(1+np.exp(-z)))
def sigmoid_der(z): return (sigmoid(z)*(1-sigmoid(z)))
#def sigmoid_der(z): return (z*(1-z))

    
    

In [119]:
bit_size = 8
sample_size = 10
epoch = 100
alpha = 0.01
max_number = np.power(2,bit_size)
a = np.array([[np.random.randint(max_number/2)] for i in range(sample_size)], dtype=np.uint8)
a_bits = np.unpackbits(a,axis=1)

b = np.array([[np.random.randint(max_number/2)] for i in range(sample_size)], dtype=np.uint8)
b_bits = np.unpackbits(b,axis=1)


c = a+b
c_bits = np.unpackbits(c,axis=1)
d = np.packbits(c_bits)

data = [[a_,b_,c_] for a_, b_, c_ in zip(a_bits,b_bits,c_bits)]

#print(a,a_bits, b,b_bits, c_bits)

In [120]:
rnn = RNN([2,3,1],data, bit_size, sample_size, epoch, alpha)

In [121]:
rnn.rnn_process()

average cost for the epoch 0 is [[0.50353559]]
average cost for the epoch 0 is [[0.50353559]]
average cost for the epoch 1 is [[0.5033499]]
average cost for the epoch 2 is [[0.50316795]]
average cost for the epoch 3 is [[0.50298967]]
average cost for the epoch 4 is [[0.50281497]]
average cost for the epoch 5 is [[0.5026438]]
average cost for the epoch 6 is [[0.50247606]]
average cost for the epoch 7 is [[0.5023117]]
average cost for the epoch 8 is [[0.50215064]]
average cost for the epoch 9 is [[0.50199281]]
average cost for the epoch 10 is [[0.50183815]]
average cost for the epoch 11 is [[0.50168659]]
average cost for the epoch 12 is [[0.50153806]]
average cost for the epoch 13 is [[0.5013925]]
average cost for the epoch 14 is [[0.50124986]]
average cost for the epoch 15 is [[0.50111006]]
average cost for the epoch 16 is [[0.50097305]]
average cost for the epoch 17 is [[0.50083876]]
average cost for the epoch 18 is [[0.50070715]]
average cost for the epoch 19 is [[0.50057816]]
average

In [93]:
data[0]

[array([0, 1, 1, 1, 0, 0, 0, 1], dtype=uint8),
 array([0, 0, 1, 1, 0, 1, 0, 0], dtype=uint8),
 array([1, 0, 1, 0, 0, 1, 0, 1], dtype=uint8)]

In [ ]:
np.sum([[1],[2],[3]])